In [3]:
import pandas as pd
import nltk
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

### Resume

In [ ]:
df_resumes = pd.read_csv('../PreProcessingJobs/processed_data/Resume_proc_lemm.csv')
df_resumes.head()

,ID,Resume_str,Category
0,16852973,hr administrator marketing associate hr admini...,HR
1,22323967,hr specialist u hr operation summary versatile...,HR
2,33176873,hr director summary year experience recruiting...,HR
3,27018550,hr specialist summary dedicated driven dynamic...,HR
4,17812897,hr manager skill highlight hr skill hr departm...,HR


In [6]:
df_resumes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          2483 non-null   int64 
 1   Resume_str  2483 non-null   object
 2   Category    2483 non-null   object
dtypes: int64(1), object(2)
memory usage: 58.3+ KB


### Job Description

In [8]:
df_jobs = pd.read_csv('../PreProcessing Jobs\processed_data\Job_proc_lemm.csv')
df_jobs.head()

,job_id,title,description,skills_desc
0,921716,Marketing Coordinator,job descriptiona leading real estate firm new ...,Requirements: \n\nWe are seeking a College or ...
1,1829192,Mental Health Therapist/Counselor,aspen therapy wellness committed serving clien...,NaN
2,10998357,Assitant Restaurant Manager,national exemplar accepting application assist...,We are currently accepting resumes for FOH - A...
3,23221523,Senior Elder Law / Trusts and Estates Associat...,senior associate attorney elder law trust esta...,This position requires a baseline understandin...
4,91700727,Economic Development and Planning Intern,job summary economic development planning inte...,NaN


In [9]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122545 entries, 0 to 122544
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   job_id       122545 non-null  int64 
 1   title        122545 non-null  object
 2   description  122545 non-null  object
 3   skills_desc  2409 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.7+ MB


### Similarity

In [10]:
tfidf_vect = TfidfVectorizer(
    lowercase=True,
    binary=False,
    max_features=10000,
    ngram_range=(1, 6),            # unigrams fino a 6-grams
    max_df=0.8,
    min_df=2
)

In [11]:
tfidf_resumes = tfidf_vect.fit_transform(df_resumes['Resume_str'])

### Test

In [12]:
#job_sample = df_jobs.sample(n=1).iloc[0]
#job_desc = job_sample['description']

In [13]:
job_sample = df_jobs[df_jobs['job_id'] == 3901356495].iloc[0] 
job_desc = job_sample['description']

In [14]:
tfidf_job = tfidf_vect.transform([job_desc])

In [15]:
similarity_vector = cosine_similarity(tfidf_job, tfidf_resumes).flatten()

In [16]:
print(similarity_vector)

[0.08951842 0.06873462 0.09289573 ... 0.07567911 0.08253002 0.06707126]


In [17]:
top_n = 5

job_real_id = job_sample['job_id']
job_title   = job_sample['title']


top_matches = similarity_vector.argsort()[::-1][:top_n]

results = []
for rank, cv_idx in enumerate(top_matches, start=1):
    cv_real_id   = df_resumes.iloc[cv_idx]['ID']
    cv_category  = df_resumes.iloc[cv_idx]['Category']
    score        = similarity_vector[cv_idx]
    
    results.append({
        'job_id':           job_real_id,
        'job_title':        job_title,
        'cv_id':            cv_real_id,
        'cv_category':      cv_category,
        'similarity_score': score,
        'rank':             rank
    })

df_matches = pd.DataFrame(results)
df_matches


,job_id,job_title,cv_id,cv_category,similarity_score,rank
0,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),12415691,DESIGNER,0.330238,1
1,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),19090468,HEALTHCARE,0.321136,2
2,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),44115326,ADVOCATE,0.289770,3
3,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),22450718,INFORMATION-TECHNOLOGY,0.276257,4
4,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),23864648,INFORMATION-TECHNOLOGY,0.259785,5


In [18]:
job_sample['description']

'mission support compute architecture operation center caoc quantum research provides program management network operation security operation technical support necessary advance service capability infrastructure technology responsibility quantum research intl quantum seeking linux administrator support team administrator standard linux system administration duty caoc huntsville al selected candidate perform standard linux system administration duty required maintain smooth operation multi user computer system consisting linux based application license server virtual machine shared service coordination network storage administrator required additional duty setting administrator service account maintaining system documentation tuning system performance installing system wide software allocate mass storage space developing monitoring policy standard allocation related use computing resource participating installation integration acceptance testing going maintenance system software environ